In [1]:
!pip install qutip

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#This File contains all the parameters used in the program.
#Physical constants, transition rates, etc.

import numpy as np 

# Universal constants
pi = np.pi
h = 6.62607015e-34  # in J s
hbar = h/(2*pi)
c = 2.99792458e10   # cm/s
kB = 1.380649e-23   # J/K
eps0 = 8.8541878128e-14    # F/cm = C/ (V cm)
# Dipolar moment
d = 3.34e-31 # C cm

#Transition energies in J (we converted the data from cm^-1 to J by multiplying by hc)
w = 9811*h*c
w1 = 237*h*c
w2 = 138*h*c
w3 = 102*h*c
w4 = 132*h*c
w5 = 150*h*c
#Spontaneous Emission in s^-1
gamma = 314.5
#Non resonant transition rates in s^-1
gamma_nr = 1e12 
#Vibronic couplings in s^-1 (k/hbar)
k_12 = 6.35e11
k_23 = 2.8e11 
k_34 = 2.25e11
k_56 = 5.4e11 
k_67 = 4.1e11 

#Refraction index
n = 1.45
#Absortion Coefficients
alpha_imp = 4e-4 # cm-1
alpha_rad = 1e-2 # cm-1
#Energy levels
E0 = 0
E1 = w1
E2 = E1 + w2
E3 = E2 + w3
E4 = E3 + w
E5 = E4 + w4
E6 = E5 + w5 
E_gs = np.array([E0,E1,E2,E3])
E_es = np.array([E4,E5,E6])

H = np.zeros(7, dtype=np.complex128)
H[0] = E0
H[1] = E1
H[2] = E2
H[3] = E3
H[4] = E4
H[5] = E5
H[6] = E6



In [5]:
def Rabifreq(j_0):
    E_0 = np.sqrt(2*j_0*1e6/(c*n*eps0))
    return d*E_0/hbar

def N(beta,omega):
    return 1/(np.exp(beta*omega)-1)
def Thermal_state(beta, H):
    state = np.exp(-beta*H)
    state[-3:] = [0,0,0]
    return state / np.sum(state)





In [16]:
def Hamiltonian(j_0_3, j_0_4):
    Omega_4 = Rabifreq(j_0_4)
    Omega_3 = Rabifreq(j_0_3)
    H = np.zeros((7,7))
    H[3,4] = Omega_4
    H[4,3] = Omega_4
    H[2,4] = Omega_3
    H[4,2] = Omega_3
    return H


In [19]:
def steady_state(T, j_0_3, j_0_4):
    beta = 1/(300*kB)
    c_nr = np.zeros((7,7))
    ws = [w1, w2, w3, w4, w4, w5]
    for i in range(6):
        c_nr[i,i+1] = np.sqrt(N(beta, ws[i])+1)
    c_nr[3,4] = 0
    c_nr = np.sqrt(gamma_nr)* c_nr

    c_nr = Qobj(c_nr)

    c_nr_dag = np.zeros((7,7))
    for i in range(1, 7):
        c_nr_dag[i,i-1] = np.sqrt(N(beta, ws[i-1]))
    c_nr_dag[4,3] = 0

    c_nr_dag = Qobj(c_nr_dag * np.sqrt(gamma_nr))

    c_se = np.zeros((7,7))
    for i in range(4):
        for j in range(4, 7):
            c_se[i,j] = 1
    c_se = np.sqrt(gamma)*c_se
    c_se = Qobj(c_se)


    H = Hamiltonian(j_0_3, j_0_4)
    H = Qobj(H) 
    rho_ss = steadystate(H, [c_nr, c_nr_dag, c_se])
    return rho_ss

In [20]:
rho_ss = steady_state(300, 0.0, 0.6)
rho_ss

Quantum object: dims = [[7], [7]], shape = (7, 7), type = oper, isherm = True
Qobj data =
[[ 5.89345715e-01+0.00000000e+00j  5.29077777e-11+0.00000000e+00j
   2.53414322e-11+0.00000000e+00j  1.31970657e-11+0.00000000e+00j
   0.00000000e+00+4.05695058e-15j  0.00000000e+00-3.25987751e-24j
   0.00000000e+00-2.10716791e-24j]
 [ 5.29077777e-11+0.00000000e+00j  1.89119526e-01+0.00000000e+00j
   3.40995813e-11+0.00000000e+00j  1.64370592e-11+0.00000000e+00j
   0.00000000e+00+1.93216596e-15j  0.00000000e+00+1.42042519e-15j
   0.00000000e+00-1.91292047e-24j]
 [ 2.53414322e-11+0.00000000e+00j  3.40995813e-11+0.00000000e+00j
   9.75672483e-02+0.00000000e+00j  2.06851975e-11+0.00000000e+00j
   0.00000000e+00+2.00668680e-15j  0.00000000e+00+1.25157743e-15j
   0.00000000e+00+5.10298897e-16j]
 [ 1.31970657e-11+0.00000000e+00j  1.64370592e-11+0.00000000e+00j
   2.06851975e-11+0.00000000e+00j  5.98210573e-02+0.00000000e+00j
   0.00000000e+00+7.21491369e-07j  0.00000000e+00+1.55131246e-15j
   0.00000000

In [21]:
print(rho_ss[4,4] + rho_ss[5,5] + rho_ss[6,6])

(0.06414645328981827+0j)
